In [0]:
dbutils.library.restartPython() 

In [0]:
import numpy as np
import pandas as pd
#!pip install typing-extensions
#!pip install sqlalchemy
from sqlalchemy import create_engine
!pip install dotenv
import dotenv 
import os
#import sqlite3
!pip install psycopg-binary
import psycopg2

In [0]:
dotenv.load_dotenv()
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
#print(POSTGRES_PASSWORD)

Anatomy of an HTML line:
* Tag: code that represents a specific visual thing to put on a page. 'a' tag is for hyperlink
<a> is our tag


In [0]:
host="ekaw-server.postgres.database.azure.com"
database="postgres"
user="ekaw_admin"
password=POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
port=5432
DBSERVER=psycopg2.connect(host=host,database=database,user=user,password=password,port=port)  
DBSERVER.autocommit = True
engine = create_engine(f"{'postgresql'}+{'psycopg2'}://{user}:{password}@{host}:{port}/{'nba'}")

In [0]:
#query this
from sqlalchemy import text
myquery = """SELECT * FROM games;"""
pd.read_sql_query(text(myquery), con=engine.connect())

In [0]:
myquery = """
SELECT * 
    FROM games
WHERE season = 2022
    AND ot > 0"""
pd.read_sql_query(text(myquery), con=engine.connect())

###Challenge 1: Who's the MVP

In [0]:
myquery = """

SELECT pg.player_id,
       p.player,
        sum(pts + fg3 - 2*fg + ft - fta + trb+ 2*ast + 4*stl - 2*tov) as mvp_points
    FROM player_game pg
INNER JOIN games g 
    ON pg.game_id = g.game_id 
INNER JOIN player p
    ON pg.player_id = p.player_id
WHERE g.season = 2022
GROUP BY pg.player_id, p.player
ORDER BY mvp_points DESC

"""

pd.read_sql_query(text(myquery), con=engine.connect())

In [0]:
myquery = """

SELECT pg.player_id,        
        SUM(CASE WHEN pts >= 10 and trb >= 10 and ast >= 10  THEN 1 ELSE 0 END) AS triple_double_indicator
    FROM player_game pg
INNER JOIN games g 
    ON pg.game_id = g.game_id 
INNER JOIN player p
    ON pg.player_id = p.player_id
WHERE g.season = 2022
GROUP BY pg.player_id
ORDER BY triple_double_indicator DESC

"""

pd.read_sql_query(text(myquery), con=engine.connect())

In [0]:
players= nba[['player_id','player']].drop_duplicates()
players.head(10)
#player_game = nba.drop(['game_date','OT','season','player']

In [0]:
team_game = nba[['game_id','Team_Abbrev','Team_Score','Team_pace','Team_efg_pct','Team_tov_pct','Team_orb_pct','Team_ft_rate','Team_off_rtg','H_A','Opponent_Abbrev']].drop_duplicates()
team_game.head(10)

In [0]:
games = nba[['game_id','game_date','OT','season']].drop_duplicates()
games.head(10)

In [0]:
player_game = nba.drop(['game_date','OT','season','player','Team_Abbrev','Team_Score','Team_pace','H_A','Team_efg_pct','Team_tov_pct','Team_orb_pct','Team_ft_rate','Team_off_rtg','Opponent_Abbrev','Opponent_Score','Opponent_pace','Opponent_efg_pct','Opponent_tov_pct','Opponent_orb_pct','Opponent_ft_rate','Opponent_off_rtg'], axis=1)
player_game.head(10)
#'Inactives'

In [0]:
#POSTGRES DOES NOT like capiutl letters in column names
player_game.columns = player_game.columns.str.lower()
games.columns = games.columns.str.lower()
team_game.columns=team_game.columns.str.lower()
players.columns= players.columns.str.lower()


###Not enough memory on my laptop to run docker, so cannot follow along beyond here with the docker parts

In [0]:
#!pip install postgresql@10


In [0]:
host="ekaw-server.postgres.database.azure.com"
database="postgres"
user="ekaw_admin"
password=POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
port=5432
DBSERVER=psycopg2.connect(host=host,database=database,user=user,password=password,port=port)  
DBSERVER.autocommit = True

In [0]:
cursor = DBSERVER.cursor()
try:    
    cursor.execute("CREATE DATABASE nba")
except:
    cursor.execute("DROP DATABASE nba")
    cursor.execute("CREATE DATABASE nba")

In [0]:
# engine = create_engine('Driver={ODBC Driver 18 for SQL Server};Server=tcp:ds6001-lab6-live-coding-pg-kaw.database.windows.net,1433;Database=DS6001_LAB6_LIVECODING_PG_KAW;Uid=ekaw_admin;Pwd={your_password_here};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;',
#                                       pool_size=10,
#                                       max_overflow=2,
#                                       pool_recycle=300,
#                                       pool_pre_ping=True,
#                                       pool_use_lifo=True)

#sqlalchemy.orm.sessionmaker(bind=engine, query_cls=RetryingQuery)

In [0]:
#use sql alchemy to create a conncetion to a database
engine = create_engine(f"{'postgresql'}+{'psycopg2'}://{user}:{password}@{host}:{port}/{'nba'}")

In [0]:
player_game.to_sql('player_game',con=engine,if_exists='replace',index=False, chunksize = 1000)
players.to_sql('player',con=engine,if_exists='replace',index=False, chunksize = 1000)
team_game.to_sql('team_game',con=engine,if_exists='replace',index=False, chunksize = 1000)
games.to_sql('games',con=engine,if_exists='replace',index=False, chunksize = 1000)

In [0]:
DBSERVER.close()